In [23]:
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import Report_SQL as RS
import cv2

In [ ]:
# # construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-m", "--model", required=True,
# 	help="path to trained serialized model")
# ap.add_argument("-l", "--label-bin", required=True,
# 	help="path to  label binarizer")
# ap.add_argument("-i", "--input", required=True,
# 	help="path to our input video")
# ap.add_argument("-o", "--output", required=True,
# 	help="path to our output video")
# ap.add_argument("-s", "--size", type=int, default=128,
# 	help="size of queue for averaging")
# args = vars(ap.parse_args())

In [24]:
cd E:/CS/Graduation Project/-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main/Face recognition

E:\CS\Graduation Project\-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main\Face recognition


In [44]:
cd E:/CS/Graduation Project/-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main/Integrated/Smoking-Dataset

E:\CS\Graduation Project\-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main\Integrated\Smoking-Dataset


In [3]:
cd E:/CS/Graduation Project/-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main/Integrated

E:\CS\Graduation Project\-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main\Integrated


In [4]:
 # load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model("CompressedBigResNet.h5")
lb = pickle.loads(open("lb.pickle", "rb").read())
# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=15) #128

[INFO] loading model and label binarizer...


In [18]:
img_counter=0

In [28]:
 # initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(0)#"pexels-mart-production-7277932.mp4"
writer = None
(W, H) = (None, None)
# loop over frames from the video file stream

while True:
	# read the next frame from the file
	#img_counter+=1
	(grabbed, frame) = vs.read()
	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break
	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]
        
# clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
	output = frame.copy()
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (200, 200)).astype("float32")
	frame -= mean
	# make predictions on the frame and then update the predictions
	# queue
	preds = model.predict(np.expand_dims(frame, axis=0))[0]
	Q.append(preds)
	# perform prediction averaging over the current history of
	# previous predictions
	results = np.array(Q).mean(axis=0)
	print (results)
	i = np.argmax(results)
	if (i==3) and results[3]<0.70000000:
		
		results[3]=0
# 	if label == "Smoking":
# 		label = "Normal"
	# draw the activity on the output frame
	i = np.argmax(results) #label="Normal" 
	label = lb.classes_[i]
	text = "activity: {}".format(label)
	cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)
	# check if the video writer is None
	im= "Detected_Frame_{}.png".format(img_counter)
	if label!="Normal":
		cv2.imwrite(im, output)
		RS.report(im,label)
	if writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		#writer = cv2.VideoWriter("tennis_128frames_smoothened.avi", fourcc, 30, (W, H), True)
	# write the output frame to disk
	#writer.write(output)
	# show the output image
	cv2.imshow("Output", output)
	key = cv2.waitKey(1) & 0xFF
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# release the file pointers
print("[INFO] cleaning up...")
#writer.release()
vs.release()
cv2.destroyAllWindows()

[0.03180271 0.2599735  0.13037853 0.5778453 ]
[0.03061852 0.2574281  0.13096188 0.58099157]
[0.02935067 0.25244325 0.13068666 0.58751947]
[0.02885848 0.24157421 0.12927781 0.6002895 ]
[0.02722773 0.23047154 0.12658142 0.6157194 ]
[0.02622197 0.22179303 0.12464014 0.6273449 ]
[0.02578425 0.21800043 0.1161722  0.6400431 ]
[0.02550497 0.21342567 0.11364077 0.6474285 ]
[0.02486034 0.20775972 0.10875256 0.65862733]
[0.02373298 0.20311971 0.10617016 0.66697717]
[0.02173267 0.19533321 0.10433856 0.6785956 ]
[0.0206181  0.1875271  0.10356253 0.6882923 ]
[0.01964966 0.18444805 0.08589198 0.71001035]
1 record inserted, ID: 8
[0.01837563 0.17205702 0.07430568 0.7352617 ]
1 record inserted, ID: 9
[INFO] cleaning up...


In [26]:
vs.release()
cv2.destroyAllWindows()